In [2]:
from microphone import record_audio

In [2]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
from IPython.display import Audio
from typing import Union, Callable, Tuple
from pathlib import Path

%matplotlib notebook

In [5]:
def librosa_resample(audio_file: np.ndarray, target_sr: int) -> np.ndarray:
    """
    Give an audio file, an original sampling rate, and a target sampling rate, returns an array for resampled audio signal
    """
    y, sr = librosa.load(audio_file, sr=target_sr)
    return y, sr

In [6]:
def quantize(samples: np.ndarray, bit_depth: int) -> np.ndarray:
    """
    Given an array of samples and a bit-depth, return the array of quantized samples.
    """
    assert bit_depth <= 14, "Exceeding this bit-depth might tank your computer!"
    quantized_values = np.linspace(samples.min(), samples.max(), 2 ** bit_depth)
    abs_differences = np.abs(samples[:, np.newaxis] - quantized_values)
    bin_lookup = np.argmin(abs_differences, axis=1)
    return quantized_values[bin_lookup]

In [7]:
def analog_to_digital_librosa(
    signal: np.ndarray,
    orig_sr: int,
    target_sr: int,
    bit_depth: int
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Digitizes a given audio signal using librosa for resampling.
    """
    resampled_signal = temporal_sampler_librosa(signal, orig_sr, target_sr)
    digital_signal = quantize(resampled_signal, bit_depth)
    times = np.linspace(0, len(signal) / orig_sr, len(digital_signal))
    return times, digital_signal